## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

### <span style="color:darkred">Módulo V - Redes neurais auto-associativas e geradoras</span>


### <span style="color:darkred">Avaliação</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

As respostas devem ser dadas no Moodle, use esse notebook apenas para gerar o código necessário para obter as respostas

---

### Questão 1)

Autoencoders do tipo Overcomplete possuem dimensão latente superior a da entrada. Como garantir o aprendizado sem que haja uma cópia simples dos dados?

(a) Utilizando mais camadas, projetando um Autoencoder Overcomplete Profundo que permita obter uma camada latente com maior qualidade<br>
(b) Substituindo a função de custo de perda ou erro quadrático pela função de custo de entropia cruzada<br>
(d) Utilizando normalização do tipo Batch ou Layer para que os dados sejam modificados ao longo da rede neural, assim evitando que haja uma cópia direta da entrada para o código<br>
<b>(d) Impondo uma restrição na projeção do código que penalize o uso de todas as dimensões do espaço latente e privilegie projeções esparsas para uma dada instância.<br></b>

---

### Questão 2)

Considere uma arquitetura com dois elementos, organizada na forma encoder-decoder. O primeiro elemento, encoder, aprende a gerar uma camada latente com os parâmetros de distribuições de probabilidade, a partir da qual o segundo componente, decoder amostra exemplos a serem reconstruídos. Esse método é conhecido por:

 (a) Continuous Generalized Autoencoder<br>
 (b) Denoising Overcomplete Autoencoder<br>
 (c) Generative Adversarial Network<br>
 <b>(d) Variational Autoencoder</b>

---
### Questão 3)

Nas redes geradoras adversariais, todos os dados de treinamento pertencem a classe positiva, enquanto que todos os dados gerados a partir de uma amostragem aleatória de uma distribuição pertence à classe negativa. É correto afirmar sobre o método de aprendizado de redes geradoras adversariais (em sua formulação original):

 (a) O modelo discriminador é um classificador, que produz uma única perda utilizada para treinar a rede como um todo. Sendo um classificador o responsável pelo ajuste dos parâmetros, consideramos o aprendizado como supervisionado.<br>
 (b) O modelo gerador produz a perda utilizada para treinar a rede como um todo. O aprendizado é não-supervisionado pelo fato de que os dados do gerador são obtidos a partir de um vetor amostrado a partir de uma distribuição aleatória.<br>
 <b>(c) O modelo discriminador (classificador) produz as perdas utilizadas para treinar a rede em associação com o gerador. Os rótulos são calculados não necessitando de anotação manual, sendo portanto não supervisionado.<br></b>
 (d) O modelo discriminador produz a perda utilizada para treinar a rede como um todo. O aprendizado é semi-supervisionado já que sabemos o rótulo das classes positivas.<br>
 


---

### Questão 4)

Carregue a base de dados `smarphone_activity_dataset.csv`, conforme abaixo, com uma divisão hold-out utilizando os 80% exemplos iniciais para treinamento e os restantes para teste.

Projete um Autoencoder para produzir uma projeção em 2 dimensões, com as seguintes camadas:
* Entrada (com as dimensões da base de dados)
* Dropout de 0.3
* Camada densa de 2 neurônios (código) e ativação linear
* Camada densa de saída (com as dimensões da base de dados) e ativação tangente hiperbólica

Inicialize as sementes `seed(1)` e `set_seed(2)` antes de instanciar o modelo, compilar e treinar.

Utilize a função de custo mean absolute error (mae), otimizador Adam com taxa 0.01, batchsize 16 e treine por 30 épocas.

Após o treinamento, obtenha um scatterplot do código de treinamento, e analise visualmente a distribuição das classes utilizando os 500 primeiros exemplos `:500`. Podemos identificar grupos com quais misturas de classes?

(a) 2 grupos, classes: (1, 3 e 6); (2, 4 e 5) <br>
<b>(b) 3 grupos, classes: (1 e 3); (2); (4, 5 e 6) <br></b>
(c) 4 grupos, classes: (1 e 3); (2); (4 e 5); (6)<br>
(d) 4 grupos, classes: (1 e 2); (3); (4); (5 e 6) <br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
import pandas as pd
from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras import models
from numpy.random import seed
#from tensorflow.random import set_seed

In [ ]:
df = pd.read_csv("smartphone_activity_dataset.csv")
df.head()

In [ ]:
rotulos = np.array(df['activity'])
features = np.array(df.iloc[:, :-1])

In [ ]:
perc_train = 0.8

n_train = int(features.shape[0]*perc_train)
n_test = int(features.shape[0]*(1-perc_train))
print(n_train)
print(n_test)

x_train = features[:n_train,:]
y_train = rotulos[:n_train]

x_test = features[n_train:,:]
y_test = rotulos[n_train:]

In [ ]:
### defina autoencoder

input_data = tf.keras.layers.Input(shape=(x_train.shape[1],))

x = keras.layers.Dropout(0.3)(input_data)
x = keras.layers.Dense(2, activation='linear')(x)
output = keras.layers.Dense(x_train.shape[1], activation='tanh')(x)

autoencoder = keras.models.Model(input_data, output)
autoencoder.summary()

In [ ]:
#sementes
seed(1)
tf.random.set_seed(2)

#epochs e batchsize
epochs = 30
batchsize = 16

autoencoder.compile(optimizer=keras.optimizers.Adam(lr=0.01),
                    loss='mae',
                    metrics=['accuracy'])

hist_autoencoder = autoencoder.fit(x_train, x_train, batch_size=batchsize, epochs=epochs) 

In [ ]:
x_sub = x_train[:500]

code = autoencoder.predict(x_sub)
fig = plt.figure(figsize=(16,9))
plt.scatter(code[:,0],code[:,1],alpha=0.5)
plt.show()

In [ ]:
import seaborn as sns

x_sub = x_train[:500,:]
extract = keras.models.Model(autoencoder.inputs, autoencoder.layers[-2].output)
code = extract.predict(x_sub)
fig = plt.figure(figsize=(16,9))
sns.scatterplot(x=code[:,0],y=code[:,1],alpha=0.5,hue=y_train[:500], palette='tab10')
plt.show()

---

### Questão 5)

Ainda utilizando a base de dados `smarphone_activity_dataset.csv` com a mesma divisão entre treinamento e teste, projete um Autoencoder profundo para produzir uma projeção em 50 dimensões, com as seguintes camadas:
* Entrada (com as dimensões da base de dados)
* 2 Camadas densas com 128 neurônios, ativação tanh
* Dropout de 0.25
* 1 Camada densa com 50 neurônios, ativação tahn, name='code'
* 2 Camadas densas com 128 neurônios, ativação tanh
* Camada densa de saída (com as dimensões da base de dados), ativação tanh

Inicialize as sementes `seed(1)` e `set_seed(2)` antes de instanciar o modelo, compilar e treinar.

Utilize a função de custo mean squared error (mse), otimizador Adam com taxa 0.001, batchsize 16 e treine por 50 épocas.

Após o treinamento, calcule o valor final da função de custo para os dados de treinamento. Obtenha o código de 50 dimensões para o conjunto de treinamento e de teste. Treine um classificador svm (utilizando o pacote `SVC` do `sklearn`), utilizando parâmetro C=0.5 e `random_state=1`, utilizando o código de treinamento, e calcule a acurácia no código do conjunto de teste.

Os valores observados de MSE de treinamento, e acurácia de classificação SVM no teste estão no intervalo:

(a) MSE =[0.10, 0.25]; Acurácia = [0.70, 0.75] <br>
(b) MSE =[0.10, 0.25]; Acurácia = [0.75, 0.77] <br>
(c) MSE =[0.01, 0.05]; Acurácia = [0.80, 0.86] <br>
<b>(d) MSE =[0.00, 0.04]; Acurácia = [0.88, 0.95] <br></b>

 **Justificativa**: Ver código abaixo.

In [ ]:
## projete e execute autoencoder
input_data = tf.keras.layers.Input(shape=(x_train.shape[1],))
x = keras.layers.Dense(128,activation='tanh')(input_data)
x = keras.layers.Dense(128,activation='tanh')(x)
x = keras.layers.Dropout(0.25)(x)
x = keras.layers.Dense(50, activation='tanh',name='code')(x)
x = keras.layers.Dense(128, activation='tanh')(x)
x = keras.layers.Dense(128, activation='tanh')(x)
output = keras.layers.Dense(x_train.shape[1],activation='tanh')(x)

autoencoder = keras.models.Model(input_data, output)
autoencoder.summary()

In [ ]:
## exiba perda de treinamento
#sementes
seed(1)
tf.random.set_seed(2)

#epochs e batchsize
epochs = 50
batchsize = 16

autoencoder.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                    loss='mse',
                    metrics=['accuracy'])

hist_autoencoder = autoencoder.fit(x_train, x_train, batch_size=batchsize, epochs=epochs) 

In [ ]:
## obtenha códigos
code_model = keras.models.Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dense_14').output)
code_train = np.asarray(code_model.predict(x_train))
code_test  = np.asarray(code_model.predict(x_test))

In [ ]:
## treine e teste classificador
from sklearn.svm import SVC

model = SVC(C = 0.5, random_state=1)
model.fit(code_train, y_train)


score = model.score(code_test,y_test)
print(score)